# transactions_hm

In [2]:
import pandas as pd
transactions = pd.read_csv("../../main/h&m_dataset/transactions_hm.csv")
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2019-11-05,3e2b60b679e62fb49516105b975560082922011dd752ec...,698328010,0.016932,2
1,2019-05-22,89647ac2274f54c770aaa4b326e0eea09610c252381f37...,760597002,0.033881,2
2,2019-05-10,2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...,488561032,0.016932,2
3,2019-08-26,7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...,682771001,0.033881,2
4,2019-08-10,3b77905de8b32045f08cedb79200cdfa477e9562429a39...,742400033,0.003220,1


## Prepare

In [3]:
transactions.info()

<class 'pandas.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   t_dat             1048575 non-null  str    
 1   customer_id       1048575 non-null  str    
 2   article_id        1048575 non-null  int64  
 3   price             1048575 non-null  float64
 4   sales_channel_id  1048575 non-null  int64  
dtypes: float64(1), int64(2), str(2)
memory usage: 40.0 MB


In [4]:
transactions.shape

(1048575, 5)

In [5]:
na_t = transactions.isna().sum().sort_values(ascending=False) 
na_t.head(3)

t_dat          0
customer_id    0
article_id     0
dtype: int64

In [6]:
transactions["sales_channel_id"].value_counts(dropna=False)

sales_channel_id
2    729192
1    319383
Name: count, dtype: int64

In [7]:
transactions.duplicated().sum()

np.int64(8474)

## 특정 고객이, 날짜에, 상품을, 가격을, 온/오프라인 에서 구매한 기록의 테이블 
고객 및 상품 테이블과의 결합을 통해 고객특성·상품군·채널 분석이 가능하다. 
- 한 행이 한 번의 구매 기록
- customer_id                   # 구매한 고객 → customers_hm과 연결
- article_id                    # 구매한 상품 → articles_hm과 연결
- t_dat                         # 구매날짜
- price                         # 구매 가격 == 매출
- sales_channel_id              # 판매 채널 구분 (1=오프라인, 2=온라인)


### 결측치 여부
- 존재하지 않음

### 형변환
- t_dat는 문자열로 들어와있기 때문에 datetime변환이 필수
- 월별, 요일별 파생 시도 가능

### 이상치 여부
- 값 대부분이 소수점으로 통화 단위가 SEK 인것 때문으로 보임
    - 이 때문에 해석의 혼동이 발생할수 있음 (따로 지정된 단위가 있는지 확인하기)
    - 컬럼명을 revenue_sek로 변경

### 중복행 여부
완전 중복행이 8474 존재 어떻게 처리해야할지 미정

### price
매출 금액은 SEK 기준 거래 단가 합으로, 절대 금액보다는 고객 간 상대적 비교에 초점을 둔다.

## 데이터 전처리

In [8]:
trans = transactions.copy()

### 날짜 컬럼 재정의

In [9]:
# 날짜 타입 형 변환
trans["t_dat"] = pd.to_datetime(
    trans["t_dat"],
    errors="coerce"
)
print("변환 완료 확인")
print(trans["t_dat"].dtype)

print("\n형 변환 실패 확인")
print(trans["t_dat"].isna().sum())

변환 완료 확인
datetime64[us]

형 변환 실패 확인
0


In [10]:
# 년 파생
trans["year"] = trans["t_dat"].dt.year

# 월 파생
trans["month"] = trans["t_dat"].dt.month

# 년월 파생
trans["year_month"] = trans["t_dat"].dt.to_period("M").astype(str)

# -------------------------------------------------------------------------
# 요일 파생 큰 의미가 없을 가능성이 있기 때문에 배제
# trans["weekday"] = trans["t_dat"].dt.day_name()
# -------------------------------------------------------------------------

trans[["year", "month", "year_month"]].value_counts()

year  month  year_month
2019  6      2019-06       122043
      7      2019-07       115505
      5      2019-05        99728
      4      2019-04        94073
      3      2019-03        81858
      1      2019-01        81248
      8      2019-08        80219
      9      2019-09        78545
      11     2019-11        76358
      10     2019-10        73820
      2      2019-02        73730
      12     2019-12        71448
Name: count, dtype: int64

### 채널 컬럼 변환

In [11]:
trans["channel"] = trans["sales_channel_id"].map({
    1: "store",
    2: "online"
}).fillna("unknown")

trans["channel"].value_counts()

channel
online    729192
store     319383
Name: count, dtype: int64

### 매출정의

In [12]:
trans.duplicated().sum()
# 중복행이긴하나 행 1개가 매출이기 때문에 인지하고 넘어가기
# 동일한 거래 정보가 완전히 일치하는 중복 행이 일부 존재하나,
# 수량 정보 부재로 인해 반복 구매 가능성을 배제할 수 없어 제거하지 않았다.

np.int64(8474)

In [13]:
trans = trans.rename(columns={"price": "revenue"})

# 고객별 총 매출 집계
customer_revenue = (
    trans.groupby("customer_id", as_index=False)
    .agg(total_revenue=("revenue", "sum"))
)

# 고객 매출 분포 기준 분위수 계산
q20 = customer_revenue["total_revenue"].quantile(0.2) # 고객 매출 분포 기준으로 하위 20% 분위수 계산
q80 = customer_revenue["total_revenue"].quantile(0.8) # 고객 매출 분포 기준으로 상위 20% 분위수 계산

In [14]:
# - bottom_20 : 하위 20%
# - top_20    : 상위 20%
# - middle    : 그 외

customer_revenue["revenue_group"] = "middle"

customer_revenue.loc[
    customer_revenue["total_revenue"] <= q20,
    "revenue_group"
] = "bottom_20"

customer_revenue.loc[
    customer_revenue["total_revenue"] >= q80,
    "revenue_group"
] = "top_20"

In [15]:
# quantile: 분위수 구함

print("하위 20% 분위수:" , q20)
print("상위 20% 분위수:", q80)

하위 20% 분위수: 0.017050847
상위 20% 분위수: 0.08977966100000001


테이블 조인

In [16]:
total_revenue = trans.merge(
    customer_revenue,
    on="customer_id",
    how="inner"
)
total_revenue = total_revenue.drop(columns=["sales_channel_id"])

# 결과 파악하기


In [23]:
print("전체 거래 수:", len(total_revenue))
print("전체 고객 수:", total_revenue["customer_id"].nunique())
print("상위 20% 고객 수:",
      customer_revenue[customer_revenue["revenue_group"] == "top_20"].shape[0])
print("하위 20% 고객 수:",
      customer_revenue[customer_revenue["revenue_group"] == "bottom_20"].shape[0])

전체 거래 수: 1048575
전체 고객 수: 458235
상위 20% 고객 수: 91860
하위 20% 고객 수: 91648


In [24]:
total_revenue.info()

<class 'pandas.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   t_dat          1048575 non-null  datetime64[us]
 1   customer_id    1048575 non-null  str           
 2   article_id     1048575 non-null  int64         
 3   revenue        1048575 non-null  float64       
 4   year           1048575 non-null  int32         
 5   month          1048575 non-null  int32         
 6   year_month     1048575 non-null  str           
 7   channel        1048575 non-null  str           
 8   total_revenue  1048575 non-null  float64       
 9   revenue_group  1048575 non-null  str           
dtypes: datetime64[us](1), float64(2), int32(2), int64(1), str(4)
memory usage: 72.0 MB


In [25]:
total_revenue.head()

,t_dat,customer_id,article_id,revenue,year,month,year_month,channel,total_revenue,revenue_group
0,2019-11-05,3e2b60b679e62fb49516105b975560082922011dd752ec...,698328010,0.016932,2019,11,2019-11,online,0.227186,top_20
1,2019-05-22,89647ac2274f54c770aaa4b326e0eea09610c252381f37...,760597002,0.033881,2019,5,2019-05,online,0.067746,middle
2,2019-05-10,2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...,488561032,0.016932,2019,5,2019-05,online,0.016932,bottom_20
3,2019-08-26,7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...,682771001,0.033881,2019,8,2019-08,online,0.033881,middle
4,2019-08-10,3b77905de8b32045f08cedb79200cdfa477e9562429a39...,742400033,0.003220,2019,8,2019-08,store,0.028627,middle


In [26]:
customer_revenue.info()

<class 'pandas.DataFrame'>
RangeIndex: 458235 entries, 0 to 458234
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   customer_id    458235 non-null  str    
 1   total_revenue  458235 non-null  float64
 2   revenue_group  458235 non-null  str    
dtypes: float64(1), str(2)
memory usage: 10.5 MB


In [27]:
customer_revenue.head()

,customer_id,total_revenue,revenue_group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.105051,top_20
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.199898,top_20
2,000064249685c11552da43ef22a5030f35a147f723d5b0...,0.042356,middle
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0.025407,middle
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.324034,top_20


# 파일저장

In [28]:
total_revenue.to_csv("data/total_revenue.csv", index=False, encoding="utf-8-sig")
customer_revenue.to_csv("data/customer_revenue.csv", index=False, encoding="utf-8-sig")

test